In [16]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from ipywidgets import interact, Dropdown

# Load shapefile
world = gpd.read_file("ne_110m_admin_0_countries.shp")

# Fix: replace names to match shapefile
country_rename = {
    "South Korea": "Republic of Korea",
    "United Kingdom": "U.K",
    "European Union": "France"
}

# Tariff datasets for April, June, July
tariffs_by_month = {
    "April": {
        "China": 34,
        "United Kingdom": 20,
        "Canada": 10,
        "Mexico": 10,
        "Japan": 24,
        "South Korea": 25,
        "India": 26,
        "Brazil": 10,
        "Australia": 10
    },
    "June": {
        "China": 34,
        "United Kingdom": 10,
        "Canada": 10,
        "Mexico": 10,
        "Japan": 10,
        "South Korea": 10,
        "India": 10,
        "Brazil": 10,
        "Australia": 10
    },
    "July": {
        "China": 125,
        "United Kingdom": 10,
        "Canada": 35,
        "Mexico": 30,
        "Japan": 15,
        "South Korea": 15,
        "India": 25,
        "Brazil": 50,
        "Australia": 10
    }
}

def plot_tariff_map(month):
    tariff_data = tariffs_by_month[month]
    tariff_df = pd.DataFrame(list(tariff_data.items()), columns=["country", "tariff"])
    tariff_df["country"] = tariff_df["country"].replace(country_rename)

    merged = world.merge(tariff_df, how="left", left_on="ADMIN", right_on="country")
    merged["ADMIN"] = merged["ADMIN"].replace({"United Kingdom": "U.K"})
    merged["centroid"] = merged.geometry.centroid

    fig, ax = plt.subplots(figsize=(20, 15))

    # Plot background
    world.dissolve(by="CONTINENT").boundary.plot(ax=ax, color='lightgray', linewidth=1)
    merged.plot(ax=ax, column="tariff", cmap="Reds", edgecolor="none",
                missing_kwds={"color": "#f7f7f7"})

    for _, row in merged.dropna(subset=["tariff"]).iterrows():
        x, y = row["centroid"].x, row["centroid"].y
        pin_length = 18
        ax.plot([x, x], [y, y + pin_length], color="red", linewidth=2)
        ax.plot(x, y + pin_length, marker="o", color="darkred", markersize=8)
        ax.text(
            x, y + pin_length + 1,
            f"{row['ADMIN']}\n{int(row['tariff'])}%",
            fontsize=14,
            ha='center',
            va='bottom',
            color='white',
            fontweight='bold',
            path_effects=[path_effects.withStroke(linewidth=2, foreground="black")],
            bbox=dict(boxstyle="round,pad=0.5", facecolor="darkred", edgecolor="black", linewidth=1)
        )

    ylim = ax.get_ylim()
    ax.set_ylim(ylim[0], ylim[1] + 20)

    ax.set_title(
        f"U.S. Tariffs ({month} 2025)",
        fontsize=32, fontweight='bold', fontname='Georgia', y=0.95
    )
    plt.suptitle(
        "Countries Affected by Newly Imposed Tariff Rates",
        fontsize=18, y=0.7, fontname='Georgia'
    )
    ax.axis("off")
    plt.tight_layout(rect=[0, 0, 1, 0.82])
    plt.show()

# 👇 Interactive dropdown in Jupyter
interact(plot_tariff_map, month=Dropdown(options=["April", "June", "July"], description="Month:"))


interactive(children=(Dropdown(description='Month:', options=('April', 'June', 'July'), value='April'), Output…

<function __main__.plot_tariff_map(month)>